https://haystack.deepset.ai/integrations/ollama

NOTE That ollama has its own embedding 

from haystack_integrations.components.embedders.ollama.document_embedder import OllamaDocumentEmbedder

from haystack_integrations.components.embedders.ollama.text_embedder import OllamaTextEmbedder




In [2]:
from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack_integrations.components.generators.ollama import OllamaGenerator


In [3]:
document_store = InMemoryDocumentStore()
document_store.write_documents(
    [
        Document(content="Super Mario was an important politician"),
        Document(content="Mario owns several castles and uses them to conduct important political business"),
        Document(
            content="Super Mario was a successful military leader who fought off several invasion attempts by "
            "his arch rival - Bowser"
        ),
    ]
)

template = """
Given only the following information, answer the question.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

LLM = 'llama3.2:1b'

pipe = Pipeline()

pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", OllamaGenerator(model=LLM, url="http://localhost:11434"))
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

query = "Who is Super Mario?"

response = pipe.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

print(response["llm"]["replies"])


['Based on the provided information, I can only answer that Super Mario is a character from a video game series.']


In [5]:
from haystack.dataclasses import ChatMessage

from haystack_integrations.components.generators.ollama import OllamaChatGenerator

messages = [
    ChatMessage.from_user("What's Natural Language Processing?"),
    ChatMessage.from_system(
        "Natural Language Processing (NLP) is a field of computer science and artificial "
        "intelligence concerned with the interaction between computers and human language"
    ),
    ChatMessage.from_user("How do I get started?"),
]
client = OllamaChatGenerator(model=LLM, timeout=45, url="http://localhost:11434")

response = client.run(messages, generation_kwargs={"temperature": 0.2})

print(response["replies"][0].text)


Getting started with Natural Language Processing (NLP) can be an exciting and rewarding experience. Here's a step-by-step guide to help you get started:

1. **Learn the basics**:
Start by understanding the fundamentals of NLP, including text processing, tokenization, stemming or lemmatization, sentiment analysis, and topic modeling.
2. **Choose a programming language**:
NLP is typically implemented in languages like Python, R, or Java. Choose a language that you're comfortable with and has good libraries for NLP tasks.
3. **Get familiar with libraries and tools**:
Some popular libraries for NLP include:
	* NLTK (Natural Language Toolkit)
	* spaCy
	* scikit-learn
	* TensorFlow or PyTorch for machine learning
4. **Learn about data structures and algorithms**:
Understand the basics of data structures like arrays, lists, and dictionaries, as well as algorithms like string matching, tokenization, and pattern recognition.
5. **Practice with datasets**:
Use publicly available datasets like IM